In [ ]:
"""
let rec termsize tm =
  match tm with
    Var x -> 1
  | Fn(f,args) -> itlist (fun t n -> termsize t + n) args 1;;
"""
def termsize(tm):
    match tm:
        case ("Var", x):
            return 1
        case ("Fn", f, args):
            return sum(map(termsize, args)) + 1


In [3]:
from dataclasses import dataclass
from typing import Tuple
class Term:
    pass
@dataclass
class Fn(Term):
    f: str
    args: Tuple[Term]

@dataclass
class Var(Term):
    x: str

def Function(fn):
    def res(*args):
        return Fn(fn, args)
    return res
def Vars(xs):
    return [Var(x) for x in xs.split()]
def Functions(fns):
    return [Function(fn) for fn in fns.split()]

import enum

# traat
# harrison
# norvig
# eli bendersky

def unify(t1,t2):


https://www.cl.cam.ac.uk/~jrh13/atp/

Knuth bendix + grobner
Can we do it? It'd be cool


In [8]:
"""
let rec lexord ord l1 l2 =
  match (l1,l2) with
    (h1::t1,h2::t2) -> if ord h1 h2 then length t1 = length t2
                       else h1 = h2 & lexord ord t1 t2
  | _ -> false;;
"""
def lexord(ord, l1, l2):
    match (l1, l2):
        case ([h1,*t1], [h2, *t2]):
            if ord(h1, h2): # yeah I dunno about this. Is this a faithful rep of ord
                return len(t1) == len(t2)
            else:
                return h1 == h2 and lexord(ord,t1, t2)
        case _:
            return False

lexord([1,5], [1,4])

False

In [ ]:
"""
let rec lpo_gt w s t =
  match (s,t) with
    (_,Var x) ->
        not(s = t) & mem x (fvt s)
  | (Fn(f,fargs),Fn(g,gargs)) ->
        exists (fun si -> lpo_ge w si t) fargs or
        forall (lpo_gt w s) gargs &
        (f = g & lexord (lpo_gt w) fargs gargs or
         w (f,length fargs) (g,length gargs))
  | _ -> false
"""
def lpo_gt(w,s,t):
    match (s,t):
        case (_, ("Var", x)):
            return s != t and x in fvt(s)
        case (("Fn", f, fargs), ("Fn", g, gargs)):
            return any(lpo_ge(w, si, t) for si in fargs) or \
                all(lpo_gt(w, s, si) for si in gargs) and \
                (f == g and lexord(lambda s,t: lpo_gt(w,s,t), fargs, gargs) or \
                 w(f, len(fargs), g, len(gargs)))
        case _:
            return False

In [ ]:
def size(t):
    match t:
        case Var(x):
            return 1
        case Fn(f, args):
            return 1 + sum(map(size, args))

def vars(t):
    match t:
        case Var(x):
            yield x
        case Fn(f, args):
            for arg in args:
                yield from vars(arg)
from collections import Counter # a multiset type

def var_comp_lte(t1,t2):
    # We don't want to have more vars on lhs than rhs
    c1 = Counter(vars(t1))
    c2 = Counter(vars(t2))
    for k in c2:
        if c1[k] < c2[k]:
            return False
    return True




    

In [ ]:
def kbo2a(t1,t2):
    if not isinstance(t2, Var):
        return False
    # check if t1 is iteerated functions symbol f over variable t2
    if not isinstance(t1, Fn):
        return False
    fn = t1.f
    def check(t):
        if isinstance(t, Var):
            return t == t2
        elif isinstance(t, Fn):
            return t.f == fn and len(t.args) == 1 and check(arg[0])
        else:
            raise ValueError("Not a term")
    return check(t1)


In [8]:
%%file /tmp/kbo_test.p
cnf(test, axiom, f = g).
cnf(false, conjecture, true = false).


Overwriting /tmp/kbo_test.p


In [9]:
!twee /tmp/kbo_test.p --precedence g,f

Here is the input problem:
  Axiom 1 (test): f = g.
  Goal 1 (false): true = false.

1. g -> f

Ran out of critical pairs. This means the conjecture is not true.
Here is the final rewrite system:
  g -> f

RESULT: CounterSatisfiable (the conjecture is false).


In [10]:
!twee /tmp/kbo_test.p --precedence f,g

Here is the input problem:
  Axiom 1 (test): f = g.
  Goal 1 (false): true = false.

1. f -> g

Ran out of critical pairs. This means the conjecture is not true.
Here is the final rewrite system:
  f -> g

RESULT: CounterSatisfiable (the conjecture is false).


In [ ]:

def kbo(t1,t2):
    if not var_comp_lte(t1,t2):
        return True
    w1 = size(t1)
    w2 = size(t2)
    if w1 > w2:
        return True
    elif w1 < w2:
        return False
    else:
         if kbo2a(t1,t2):
             return True
         elif t2.f < t1.f:   #kbo2b
            return True
        elif: #kbo2c
            
        else:
            return None

In [ ]:
%%bash

dedu

https://link.springer.com/content/pdf/10.1007/3-540-17220-3_4.pdf How to choose the weights in the Knuth Bendix ordering -- ursula martin

https://academic.oup.com/comjnl/article/34/1/2/427931 An Introduction to Knuth–Bendix Completion Ajj dick

https://link.springer.com/article/10.1007/s10817-006-9031-4  Things to Know when Implementing KBO

We have a notion that simplifiers must terminate because terms get simpler until they can't

"size" is a basic notion of complexity.

In order to guarantee a rule with variables gets smaller, we can count the number of symbols in it. The number of pattern variables has to be getting smaller, or else the thing is getting bigger.

`(X + 0) -> X` makes the term simpler

`foo(bar(1,2,3,4,5,6,X)) -> biz(X,X)` is not definitely a simplification, because X might be huge. In a realistic situation, we might want to guard this rule by `size(X) < 3` or something.

A slight extension is to give the symbols in the term weight.

Another example is associating to the right. X + (Y + Z) -> (X + Y) + Z. This also obviously terminates. We can give lexicographic preference to left subtrees.


Another example is getting flatter. The height is descreasing.

distrbutivity is another common case. x*(y + z) -> x*y + x*z. This is getting bigger _and_ duplicating x. That's a big nono for most term orderings. But clearly foiling is terminating.

A different system that is obviously terminating is stratified macro expansion. Macros can refer to previously defined macros. The terms however, may get very large and be increasing in size.
f(X) -> bar(X,X) is not an issue.
Cycles in the dependency graph

We don't expect to be able to have a total ordering on terms with variables in them. But we do want our definition to have a well defined

A primitive system that is obviously terminating are systems of the form `f(x,y,z,...) -> z` This is a view on the homeomorphic embedding relation.

right ground systems

Many functional programs are "obviously" terminating even if we don't think of them in these terms

Negation normal form pushing



In [ ]:
# group axioms
[0 + x == x,
 x + 0 == x,
 -x + x == 0,
 (x + y) + z == x + (y + z)]

# inv(inv(x)) == x
prove(-(-(x)) == x)

What about naive breadth first search:
Separate into confluent terminating rules
and naughty rules. (maude makes this disctinction but do so dynamically)

Hash cons only things that are normalized with respect to current rules.
run twee for a bit, extract oriented rules, confluence them (?)
Use the others for search.

TRS/SK90 Joachim Steinbach, Ulrich Kühler: Check your Ordering - termination proofs and open Problems, Technical Report SR-90-25, Universität Kaiserslautern, 1990.
TRS/D33 Nachum Dershowitz: 33 Examples of Termination, 1995 Proc. French Spring School of Theoretical Computer Science, LNCS 909, http://www.math.tau.ac.il/~nachumd/papers/printemp-print.pdf
TRS/AG01 Thomas Arts, Jürgen Giesl: Termination of term rewriting using dependency pairs, 2000, http://dblp.uni-trier.de/rec/bibtex/journals/tcs/ArtsG00 http://verify.rwth-aachen.de/giesl/papers/ibn-97-46.ps
SRS/Zantema 128 string rewriting termination problems collected by Hans Zantema (2004?). They include (as z027 .. z064) a set of one-rule termination problems by Alfons Geser (Habilitationsschrift, Tübingen, 2001) and possibly Winfried Kurth (Dissertation, Clausthal, 1990)

disjunctive normal form example. Makes term bigger.




In [ ]:
# naive

def rw(t):
    match t:
        case ("+", 0, x):
            return x
        case x: 
            return ("+", 0, x)
        case ("+", x, 0):
            return x
        case x:
            return ("+", x, 0)
        case ("+", ("-", x), x):
            return 0
        
        


def expand(t):
    for (lhs,rhs) in rules:
        yield from apply(lhs,rhs,t)
        yield from apply(rhs,lhs,t)

def prove(lhs,rhs):
    lhsseen = set(lhs)
    rhsseen = set(rhs)
    while lhsseen & rhsseen == set():
        for t in expand(lhsseen):
            if t in rhsseen:
                return True
        for t in expand(rhsseen):
            if t in lhsseen:
                return True
    


